In [1]:
import pandas as pd
import numpy as np
import math
from itertools import combinations
from statsmodels.tsa.statespace.sarimax import SARIMAX
from metrics import *
from typing import Callable
import matplotlib.pyplot as plt

In [2]:
def transf(x):
    return np.sqrt(x)
def itransf(x):
    return np.square(x)

In [3]:
# import pandas as pd
# import numpy as np
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from typing import Callable

def forecast_SARIMAX(window: int, n_train: int, p: int, d: int, q: int, ps: int, ds: int, qs: int, m: int, 
                     x: list[str], y: str, h_max: int, transf: Callable[[float], float], itransf: Callable[[float], float] )-> pd.DataFrame:
    
    """
    This function uses the SARIMAX model from statsmodels. It predicts 'y' by using 'x' as exogenous variables. 
    For each exogenous variable, the function takes 1 lag at a distance of 'h' time units.
    It forecasts 'h_max' time steps in future.

    Arguments
    ----------
    window : int
        window number
    n_train : int
        the size of training set
    p : int
        the value for auto regression order
    d : int
        the value for differencing 
    q : int
        the value for moving average order
    ps : int
        the value for seasonal auto regression order
    ds : int
        the value for seasonal differencing 
    qs : int
        the value for seasonal moving average order
    m : int
        the number of time steps for a single seasonal period
    x : list[str]
        the list of exogenous variables
    y : str
        the forecasted variable
    h_max : int
        the maximum number of forecasted horizons
    transf: Callable[[float], float]
        the definition for transformation
    itransf: Callable[[float], float]
        the definition for inverse transformation

    Returned Values
    ----------
    forecast_f : pd.DataFrame
      DataFrame containing the forecasted variable for 'h_max' horizons.

    """

    df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    date = df_US['date']
    date_list = date.tolist()
    df_US = df_US.drop(columns=['date'])
    
    #Apply transformation
    df_US_transf = pd.DataFrame()
    for col in df_US.columns:
        df_US_transf[col] = df_US[col].apply(lambda x: transf(x))


    #List of exogenous variables
    total_features = ['reproduction_rate','icu_patients', 'hosp_patients','positive_rate','new_cases','new_tests' , 'people_vaccinated', 'people_fully_vaccinated']

    df_y = df_US_transf[y]
    df_x = df_US_transf[total_features]
    value = np.empty(h_max)
    index = list()
    #Predicting weeks in future according to the horizon (h) 
    for h in range(1,h_max+1):

    #Shifting exogenous variables according to the horizon (h)   
        x_lagged_variables = df_x.shift(h).bfill()
        exog_variables = x_lagged_variables[x]

        # Its a moving window that starts from the window value till the train size plus the 'h' time steps in future
        df_window = df_y[int(window-1):int(window-1+n_train+h)]

        i = h-1
        
        #Defining train and test data
        col_train = df_y.loc[df_window.index][0:n_train]
        exo_train = exog_variables.loc[df_window.index][0:n_train]
        exo_test = exog_variables.loc[df_window.index][n_train:]

        #Model definition for prediction
        model = SARIMAX(col_train, exog=exo_train, order=(p,d,q), seasonal_order=(ps,ds,qs,m), enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        forecast = model_fit.predict(len(col_train), len(col_train)+i, exog=exo_test)

        #Inverse transformation
        forecast = itransf(forecast)
        forecast_f = forecast.to_frame()

        #For each horizon, the function selectively uses values that have a lag/shift equal to the specific horizon
        if h == 1:
            df1 = forecast_f.rename(columns= {0: y})
            value[i] = df1[y].iloc[i]
            index.append(date_list[n_train+window-1])            

        else:
            df2 = forecast_f.rename(columns= {'predicted_mean': y})
            value[i] = df2[y].iloc[i]
            index.append(date_list[n_train+window-2+h])
            
    data = {'index': index, y: value}
    forecast_f = pd.DataFrame(data)
    forecast_f['index'] = pd.to_datetime(forecast_f['index'])

    # Set the 'index' column as the DataFrame's index
    forecast_f.set_index('index', inplace=True)
    
    return forecast_f

In [4]:
def window_results(n_train: int, n_test: int, p: int, d: int, q: int, ps: int, ds: int, qs: int, m: int, 
                      x: list[str], y: str, h_max: int, transf ,itransf)-> pd.DataFrame:
     
    """
    A function that returns the weekly prediction for each window. 

    Arguments
    ----------
    n_train : int
        the size of training set
    n_test : int
        the size of testing set
    p : int
        the value for auto regression order
    q : int
        the value for moving average order
    ps : int
        the value for seasonal auto regression order
    qs : int
        the value for seasonal moving average order
    w : int
        the number of time steps for a single seasonal period
    d : int
        the value for differencing 
    ds : int
        the value for seasonal differencing 
    exo : list[str]
        the list of exogenous variables
        
    Returned Values
    ----------
    temp_results : pd.DataFrame
        A DataFrame containing weekly predictions for each window

    """  
    temp_results = pd.DataFrame()
    for i in range(n_test):
        window = i + 1
        predict_current = forecast_SARIMAX(window, n_train, p, d, q, ps, ds, qs, m, x, y, h_max, transf, itransf)
        predict_current = predict_current.rename(columns={str(y):"window_"+str(window)})
        temp_results = pd.concat([temp_results, predict_current], axis=1)
#         print(temp_results)
    return temp_results

In [5]:
def output_h(temp_results: pd.DataFrame, h: int, n_test:int)-> pd.DataFrame:
    """
    This function processes the 'temp_results' DataFrame, which contains predictions for h weeks ahead for each window, and organizes the data into a single column for each horizon.

    Arguments
    ----------
    temp_results : pd.DataFrame
        the dataframe that has h weeks ahead prediction for each window
    h : int
        horizon value
    n_test : int
        the size of testing set

    Returned Values
    ----------
    df_results_python : pd.DataFrame
        Dataframe for each horizon.  

    """ 
    n_iter = n_test-h+1
    df_results = []
    if h==1:
        for i in range(int(n_iter)):
          window = i + 1
          current_results = temp_results[['window_'+str(window)]].dropna()
          current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
          df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
    if h>1:
        for i in range(int(h-1)):
          window = i + 1
          current_results = temp_results[['window_'+str(window)]].dropna()
          current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
          df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())

        for i in range(int(n_iter)):
          window = i + 1
          current_results = temp_results[['window_'+str(window)]].dropna()
          current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
          df_results.append(pd.DataFrame(current_results.iloc[int(h-1)]).transpose())
    df_results_python = pd.concat(df_results,axis=0)
    return df_results_python

In [6]:
def predict_table(n_test, temp_results)-> pd.DataFrame:
    """
    A function that iterates through each horizon and combines the predictions for each horizon into a final DataFrame that has 'h' columns for each week.

    Arguments
    ----------
    n_test : int
        the size of testing set
    temp_results : pd.DataFrame
        the dataframe that has h weeks ahead prediction for each window

    Returned Values
    ----------
    df_final_forecast : pd.DataFrame
        DataFrame presenting the h weeks ahead prediction for each week.

    """ 
    df_final_forecast = pd.DataFrame()
    for h in range(1,5):
        df_current = output_h(temp_results, h, n_test)
        df_final_forecast = pd.concat([df_final_forecast, df_current], axis=1)
    return df_final_forecast

In [7]:
def smape_results(df_final_forecast: pd.DataFrame ,y: str):
    """ 
    This function calculates the sMAPE score using the sMAPE definition between the original data and the predictions for each horizon
    
    Arguments
    ----------
    df_final_forecast: pd.DataFrame
        the predicted DataFrame containing h weeks ahead prediction for each week
    p : int
        the value for auto regression order

    Returned Values
    ----------
    df_results : pd.DataFrame
        DataFrame with SMAPE scores for each horizon and the average SMAPE score.
        
    """  
    df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    h_level = df_final_forecast.columns.to_list()
    eval = []
    for cols in h_level:
        y_true = df_US.loc[df_final_forecast.index][y]
        y_pred = df_final_forecast[cols]
        value = smape(y_true, y_pred)
        eval.append(value)
    eval.append("{:.2f}".format(np.mean(eval)))
    df_results = pd.DataFrame(eval, columns=[str(y)])
    h_level.append('Average')
    df_results.index = h_level
    return df_results

In [8]:
def mae_results(df_final_forecast, y):
  df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  #df_predict = np.exp(df_final_forecast)
  h_level = df_final_forecast.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_final_forecast.index][y]
    y_pred = df_final_forecast[cols]
#     print(cols)
#     plt.figure(figsize=(10, 6))
#     plt.plot(df_US[col_core], label='True')
#     plt.plot(df_final_forecast[cols], "-o", label='Predicted')
#     plt.xlabel('Date')
#     plt.ylabel('Number of Deaths')
#     plt.title('SARIMAX Prediction vs. Test Data')
#     plt.legend()
#     plt.show()
    value = mae(y_true, y_pred)
    eval.append(value)
  eval.append("{:.2f}".format(np.mean(eval)))
  df_results = pd.DataFrame(eval, columns=[str(y)])
  h_level.append('Average')
  df_results.index = h_level

  return df_results

In [9]:
def rmse_results(df_final_forecast, y):
    """ 
    This function calculates the sMAPE score using the sMAPE definition between the original data and the predictions for each horizon
    
    Arguments
    ----------
    df_final_forecast: pd.DataFrame
        the predicted DataFrame containing h weeks ahead prediction for each week
    p : int
        the value for auto regression order

    Returned Values
    ----------
    df_results : pd.DataFrame
        DataFrame with SMAPE scores for each horizon and the average SMAPE score.
        
    """  
    df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    h_level = df_final_forecast.columns.to_list()
    eval = []
    for cols in h_level:
        y_true = df_US.loc[df_final_forecast.index][y]
        y_pred = df_final_forecast[cols]
        value = rmse(y_true, y_pred)
        eval.append(value)
    eval.append(np.mean(eval))
    df_results = pd.DataFrame(eval, columns=[str(y)])
    h_level.append('Average')
    df_results.index = h_level
    return df_results

In [10]:
def relative_absolute_error(true, pred):
    true_mean = np.mean(true)
    squared_error_num = np.sum(np.abs(true - pred))
    squared_error_den = np.sum(np.abs(true - true_mean))
    rae_loss = squared_error_num / squared_error_den
    return rae_loss

In [11]:
def mrae_results(df_final_forecast,y):
  col_core = 'new_deaths'
  df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  # df_predict = np.exp(df_final_forecast)
  h_level = df_final_forecast.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_final_forecast.index][col_core]
    y_pred = df_final_forecast[cols]
    value = relative_absolute_error(y_true, y_pred)
    eval.append(value)
  eval.append("{:.2f}".format(np.mean(eval)))
  df_results = pd.DataFrame(eval, columns=[str(y)])
  h_level.append('Average')
  df_results.index = h_level
  return df_results

In [12]:
#For reproducibility cdc results
import warnings
warnings.filterwarnings('ignore')
n_train = 40
n_test = 79
# n_train = 28
# n_test = 91
y = 'new_deaths'
m = 10
p = 1
ps = 1
qs = 0
d = 1
q = 1
ds = 0
h_max = 4
x = ['icu_patients',  'positive_rate']
temp_results = window_results(n_train, n_test, p, d, q, ps, ds, qs, m, x, y, h_max, transf,itransf)
df_final_forecast = predict_table(n_test, temp_results)
df_smape_results = smape_results(df_final_forecast, y)
print("sMAPE {}".format(df_smape_results))
df_mae_results = mae_results(df_final_forecast, y)
print("MAE {}".format(df_mae_results))
df_mrae_results = mrae_results(df_final_forecast,y)
print("MRAE {}".format(df_mrae_results))

sMAPE         new_deaths
h=1      11.005245
h=2      12.875147
h=3      17.897653
h=4      25.955123
Average      16.93
MAE           new_deaths
h=1       887.674456
h=2      1000.879661
h=3      1369.283169
h=4       2086.94508
Average      1336.20
MRAE         new_deaths
h=1       0.170247
h=2       0.191958
h=3       0.262615
h=4       0.400255
Average       0.26


In [13]:
df_final_forecast

,h=1,h=2,h=3,h=4
2020-12-05,11309.483557,11309.483557,11309.483557,11309.483557
2020-12-12,14594.575997,12181.646805,14594.575997,14594.575997
2020-12-19,17159.258316,14945.926731,13684.646888,17159.258316
2020-12-26,19216.127278,19197.560110,14964.305629,15528.946168
2021-01-02,19040.335032,19479.211533,17743.474009,18439.517889
...,...,...,...,...
2022-05-07,2303.321100,3226.069753,4094.510391,4722.424702
2022-05-14,2537.256361,2850.547483,3935.536220,5387.085390
2022-05-21,2238.149254,3045.142320,3355.673830,4566.580671
2022-05-28,2411.214087,2942.395919,3632.643067,3751.888065


In [14]:
# #For reproducibility cdc results
# import warnings
# warnings.filterwarnings('ignore')
# n_train = 40
# n_test = 79
# y = 'new_deaths'
# m = 10
# p = 1
# ps = 1
# qs = 0
# d = 1
# q = 1
# ds = 0
# h_max = 4
# all_features = ['icu_patients', 'hosp_patients', 'positive_rate','new_cases','new_tests','people_vaccinated', 'people_fully_vaccinated','reproduction_rate']
# for r in range(1,len(all_features)+1):
#     new_feature_list = list(combinations(all_features,r))
#     for items in new_feature_list:
#         x = list(items)
#         print("Exogenous features:" , x)
#         temp_results = window_results(n_train, n_test, p, d, q, ps, ds, qs, m, x, y, h_max, transf,itransf)
#         df_final_forecast = predict_table(n_test, temp_results)
#         df_smape_results = smape_results(df_final_forecast, y)
#         print("sMAPE {}".format(df_smape_results))
#         df_mae_results = mae_results(df_final_forecast, y)
#         print("MAE {}".format(df_mae_results))
#         df_mrae_results = mrae_results(df_final_forecast,y)
#         print("MRAE {}".format(df_mrae_results))

In [15]:
def plot_model(df_final_forecast, df_US_CDC, model):
    common_dates = df_final_forecast.index.intersection(df_US_CDC.index)
    df_final_forecast_aligned = df_final_forecast.loc[common_dates]
    col_core = 'new_deaths'
    df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    h_level = df_US_CDC.columns.to_list()
    for col in h_level:
        y_true = df_US.loc[df_US_CDC.index][col_core]
        y_pred_CDC = df_US_CDC[col]
        y_pred_SARIMAX = df_final_forecast_aligned[col]
        plt.figure(figsize=(10, 6))
        plt.plot(df_US.index,df_US[col_core], label='True')
        plt.plot(df_final_forecast_aligned.index, y_pred_SARIMAX, label='Predicted_SARIMAX', color='purple')
        if model is not None:
            plt.plot(df_US_CDC.index, y_pred_CDC, label='Predicted_'+model)
            plt.title(model+' '+col)
        else:
            plt.title('SARIMAX Prediction vs. Test Data')
        plt.xlabel('Date')
        plt.ylabel('New Deaths')
        plt.legend()
        plt.show()

In [28]:
def calculate_smape(df_final_forecast, df_US_CDC, model):
    common_dates = df_final_forecast.index.intersection(df_US_CDC.index)
    dates_not_present = df_final_forecast.index.difference(df_US_CDC.index)
    print(dates_not_present)
    df_final_forecast_aligned = df_final_forecast.loc[common_dates]
    col_core = 'new_deaths'
    df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    h_level = df_final_forecast_aligned.columns.to_list()
    eval = []
    for cols in h_level:
        y_true = df_US.loc[df_final_forecast_aligned.index][y]
        y_pred = df_final_forecast_aligned[cols]
        value = smape(y_true, y_pred)
        eval.append(value)
    eval.append("{:.2f}".format(np.mean(eval)))
    df_results = pd.DataFrame(eval, columns=[str(y)])
    h_level.append('Average')
    df_results.index = h_level
    print(df_results)

In [29]:
def calculate_rmae(df_final_forecast, df_US_CDC, model):
    common_dates = df_final_forecast.index.intersection(df_US_CDC.index)
    df_final_forecast_aligned = df_final_forecast.loc[common_dates]
    print(df_final_forecast_aligned)
    col_core = 'new_deaths'
    df_US = pd.read_csv('2023-07-29-20-58-15-OWID_weekly_updated.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    h_level = df_final_forecast_aligned.columns.to_list()
    eval = []
    for cols in h_level:
        y_true = df_US.loc[df_final_forecast_aligned.index][y]
        y_pred = df_final_forecast_aligned[cols]
        value = relative_absolute_error(y_true, y_pred)
        eval.append(value)
    eval.append("{:.2f}".format(np.mean(eval)))
    df_results = pd.DataFrame(eval, columns=[str(y)])
    h_level.append('Average')
    df_results.index = h_level
    print(df_results)

In [30]:
def cdc_data(model):
    df_US_CDC = pd.read_csv('concatenated_CDC_20_21_22_23.csv', on_bad_lines='skip')
    df_US_CDC = df_US_CDC[(df_US_CDC['Model'] == model)]
    df_US_CDC = df_US_CDC[['Date Model was run', 'Point']]
    df_US_CDC['Date Model was run'] = pd.to_datetime(df_US_CDC['Date Model was run'])
    df_US_CDC = df_US_CDC.sort_values('Date Model was run').groupby('Date Model was run')['Point'].apply(lambda df: df.reset_index(drop=True)).unstack().reset_index()
    # print(df_US_CDC)
    df_US_CDC.set_index('Date Model was run', inplace=True)
    df_US_CDC.columns =['h=1', 'h=2', 'h=3', 'h=4']
    pd.set_option('display.max_rows', 10)
    start_date = pd.to_datetime('2020-09-12')
    end_date = pd.to_datetime('2022-06-04')
    df_US_CDC_1 = df_US_CDC.loc[start_date:end_date]
    print(model)
    print(df_US_CDC_1)
    df_smape_results = smape_results(df_US_CDC_1,"new_deaths")
    df_smape_results.columns = ['SMAPE'] * len(df_smape_results.columns)
    print("{}".format(df_smape_results))
    # df_mae_results = mae_results(df_US_CDC,1)
    # df_mae_results.columns = [''] * len(df_mae_results.columns)
    # print("{}".format(df_mae_results.tail(1)))
    df_mrae_results = mrae_results(df_US_CDC_1,"new_deaths")
    df_mrae_results.columns = ['MRAE'] * len(df_mrae_results.columns)
    print("{}".format(df_mrae_results))
    print("####################")
    #plot_model(df_final_forecast, df_US_CDC_1, model)
#     calculate_smape(df_final_forecast, df_US_CDC_1, model)
#     calculate_rmae(df_final_forecast, df_US_CDC_1, model)

In [31]:
cdc_data("MIT-LCP")

MIT-LCP
                     h=1   h=2   h=3   h=4
Date Model was run                        
2020-09-12          5164  3144  3390  4424
2020-09-19          4833  4738  4034  3189
2020-09-26          4533  4684  4405  3900
2020-10-10          4178  3639  4465  4474
2020-10-17          5010  5271  4808  4568
...                  ...   ...   ...   ...
2022-04-30          3023  3083  3087  3157
2022-05-14          2608  2602  2545  2523
2022-05-21          2244  2171  2237  2231
2022-05-28          2360  2351  2382  2384
2022-06-04          2120  2091  2121  2085

[77 rows x 4 columns]
             SMAPE
h=1      16.168889
h=2      16.836542
h=3      14.867271
h=4      17.078551
Average      16.24
             MRAE
h=1      0.276674
h=2      0.298486
h=3      0.260205
h=4      0.300744
Average      0.28
####################
DatetimeIndex(['2021-01-23', '2021-08-14', '2021-09-25', '2021-10-02',
               '2021-11-13', '2021-12-04', '2022-01-15', '2022-05-07'],
              dtype='dat

In [ ]:
cdc_data("BPagano")
cdc_data("Columbia")
cdc_data("MIT-LCP")
cdc_data("CovidComplete")
cdc_data("ESG")
cdc_data("GT-DeepCOVID")
cdc_data("JHU-APL")
cdc_data("Karlen")
cdc_data("MIT-ORC")
cdc_data("MOBS")
cdc_data("PSI")
cdc_data("UCSD-NEU")
cdc_data("UM")

In [ ]:

cdc_data("Covid19Sim")

cdc_data("IHME")

cdc_data("JHU-IDD")

cdc_data("Microsoft")
cdc_data("MIT-ISOLAT")

cdc_data("QJHong")

cdc_data("UMass-MB")

# # cdc_data("Alpert")
cdc_data("Columbia-UNC")
cdc_data("DDS")
cdc_data("IEM")
cdc_data("ISU")
cdc_data("JCB")
cdc_data("JHU-CSSE")
cdc_data("LANL")
cdc_data("LUcompUncertLab")
cdc_data("Masaryk")
cdc_data("NotreDame-Mobility")
cdc_data("Oliver-Wyman")
cdc_data("RPI-UW")
cdc_data("UA")
cdc_data("UCLA")
cdc_data("UCM")
cdc_data("UGA-CEID")
cdc_data("UpstateSU")
cdc_data("UT")